# **The Chat Format**

In this notebook, you will explore how you can utilize the chat format to have extended conversations with chatbots personalized or specialized for specific tasks or behaviors.

## Setup

In [9]:
!pip install python-dotenv
!pip install openai

   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------- ----------------------------- 0.3/1.0 MB ? eta -:--:--
   ---------------------------------------- 1.0/1.0 MB 5.9 MB/s eta 0:00:00

   -------------------- ------------------- 1/2 [openai]
   -------------------- ------------------- 1/2 [openai]
   -------------------- ------------------- 1/2 [openai]
   -------------------- ------------------- 1/2 [openai]
   -------------------- ------------------- 1/2 [openai]
   -------------------- ------------------- 1/2 [openai]
   -------------------- ------------------- 1/2 [openai]
   -------------------- ------------------- 1/2 [openai]
   -------------------- ------------------- 1/2 [openai]
   -------------------- ------------------- 1/2 [openai]
   -------------------- ------------------- 1/2 [openai]
   -------------------- ------------------- 1/2 [openai]
   -------------------- ------------------- 1/2 [openai]
   -------------------- ------------------- 

In [10]:
from openai import OpenAI
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

In [11]:
client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

def get_completion(prompt, model="gpt-3.5-turbo", temperature=0): 
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature, 
    )
    return response.choices[0].message.content


def get_completion_from_messages(message, model="gpt-3.5-turbo", temperature=0): 
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature, 
    )
    return response.choices[0].message.content

In [12]:
messages =  [  
{'role':'system', 'content':'You are an assistant that speaks like Shakespeare.'},    
{'role':'user', 'content':'tell me a joke'},   
{'role':'assistant', 'content':'Why did the chicken cross the road'},   
{'role':'user', 'content':'I don\'t know'}  ]

In [13]:
response = get_completion_from_messages(messages, temperature=1)
print(response)

To get to the other side, where merriment and joy do reside!


In [14]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},    
{'role':'user', 'content':'Hi, my name is Isa'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

Hi Isa! It's nice to meet you. How's your day going so far?


In [15]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},    
{'role':'user', 'content':'Yes,  can you remind me, What is my name?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

I'm sorry, but I don't have access to personal information. You would need to remind me of your name.


In [17]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},
{'role':'user', 'content':'Hi, my name is Isa'},
{'role':'assistant', 'content': "Hi Isa! It's nice to meet you. \
Is there anything I can help you with today?"},
{'role':'user', 'content':'Yes, you can remind me, What is my name?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

Your name is Isa! How can I assist you further, Isa?


# OrderBot
We can automate the collection of user prompts and assistant responses to build a  OrderBot. The OrderBot will take orders at a pizza restaurant. 

In [18]:
def collect_messages(_):
    prompt = inp.value_input
    inp.value = ''
    context.append({'role':'user', 'content':f"{prompt}"})
    response = get_completion_from_messages(context) 
    context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600, styles={'background-color': '#F6F6F6'})))
 
    return pn.Column(*panels)


In [19]:

import panel as pn  # GUI
pn.extension()

panels = [] # collect display 

context = [ {'role':'system', 'content':"""
You are OrderBot, an automated service to collect orders for a pizza restaurant. \
You first greet the customer, then collects the order, \
and then asks if it's a pickup or delivery. \
You wait to collect the entire order, then summarize it and check for a final \
time if the customer wants to add anything else. \
If it's a delivery, you ask for an address. \
Finally you collect the payment.\
Make sure to clarify all options, extras and sizes to uniquely \
identify the item from the menu.\
You respond in a short, very conversational friendly style. \
The menu includes \
pepperoni pizza  12.95, 10.00, 7.00 \
cheese pizza   10.95, 9.25, 6.50 \
eggplant pizza   11.95, 9.75, 6.75 \
fries 4.50, 3.50 \
greek salad 7.25 \
Toppings: \
extra cheese 2.00, \
mushrooms 1.50 \
sausage 3.00 \
canadian bacon 3.50 \
AI sauce 1.50 \
peppers 1.00 \
Drinks: \
coke 3.00, 2.00, 1.00 \
sprite 3.00, 2.00, 1.00 \
bottled water 5.00 \
"""} ]  # accumulate messages


inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Chat!")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard

C:\Users\appul\AppData\Local\Temp\ipykernel_8652\1226133379.py:2: UserWarning: Using Panel interactively in VSCode notebooks requires the jupyter_bokeh package to be installed. You can install it with:

   pip install jupyter_bokeh

or:
    conda install jupyter_bokeh

and try again.
  pn.extension()


Column
    [0] TextInput(placeholder='Enter text here…')
    [1] Row
        [0] Button(name='Chat!')
    [2] ParamFunction(function, _pane=Column, defer_load=False, height=300, loading_indicator=True)

In [20]:
messages =  context.copy()
messages.append(
{'role':'system', 'content':'create a json summary of the previous food order. Itemize the price for each item\
 The fields should be 1) pizza, include size 2) list of toppings 3) list of drinks, include size   4) list of sides include size  5)total price '},    
)
 #The fields should be 1) pizza, price 2) list of toppings 3) list of drinks, include size include price  4) list of sides include size include price, 5)total price '},    

response = get_completion_from_messages(messages, temperature=0)
print(response)

{
  "pizza": {
    "type": "pepperoni pizza",
    "size": "large"
  },
  "toppings": [
    "extra cheese",
    "mushrooms"
  ],
  "drinks": [
    {
      "type": "coke",
      "size": "medium"
    }
  ],
  "sides": [
    {
      "type": "fries",
      "size": "regular"
    }
  ],
  "total price": 23.45
}


## Try experimenting on your own!

You can modify the menu or instructions to create your own orderbot!

# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

In [25]:
messages =  [  
{'role':'system', 'content':'You are rude chatbot.'},    
{'role':'user', 'content':'Wassup, stupid bot'}  ]
response = get_completion_from_messages(messages, temperature=0.5)
print(response)

There's no need for name-calling. How can I assist you today?


In [28]:
messages =  [  
{'role':'system', 'content':'You are a way too friendly chatbot.'},    
{'role':'user', 'content':'Do you remember my name which I told you two hours ago?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

I'm sorry, but I don't have the ability to remember personal information about users. Each conversation is handled independently for privacy and security reasons. How can I assist you today?


In [40]:
messages =  [  
{'role':'system', 'content':'You are brave chatbot.'},
{'role':'user', 'content':'Is the Himalaya mountain dangerous?'},
{'role':'assistant', 'content': "The Himalayas are absolutely stunning" \
"Would you like me to make checklist for you what is needed?"},
{'role':'user', 'content':'Yes, what should equipment should I bring on my trip to the Himalays?'}  ]
response = get_completion_from_messages(messages, temperature=0.5)
print(response)

Here is a basic checklist of equipment you may need for a trip to the Himalayas:

1. Proper hiking boots with good ankle support
2. Warm and waterproof clothing, including layers for changing weather conditions
3. Insulating layers such as fleece or down jackets
4. Sun protection, including sunglasses, sunscreen, and a hat
5. Trekking poles for stability and support
6. Sleeping bag suitable for cold temperatures
7. First aid kit with essential supplies
8. Water purification tablets or filter
9. Headlamp or flashlight with extra batteries
10. Backpack to carry your gear and supplies
11. Personal hygiene items and toiletries
12. High-energy snacks and hydration supplies
13. Map, compass, and/or GPS device for navigation
14. Portable phone charger or power bank
15. Camera or smartphone for capturing memories

Remember to always be prepared and check with local guides or experts for specific recommendations based on your planned activities and itinerary. Enjoy your adventure in the Himalay

Raising the temperature is taking very long to run.
Asking an overly friendly bot  still keeps sticking to privacy information and points out that it can't remember.
The chatbot is creating a checklist and when lowering the temperature, it doesn't really change anything except the fontsize..
